# ECE 4300 Not Hotdog

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.io
import skimage.transform
import urllib
import requests
from io import BytesIO
import flickrapi

In [ ]:
api_key = '8fa193368446fcb9fbedaea28db13ccc'
api_secret = 'e032c5956a398644'

flickr = flickrapi.FlickrAPI(api_key, api_secret)